In [1]:
import pandas as pd 
from urllib.parse import urlencode
import csv
import numpy as np 
import re, itertools
from top2vec  import Top2Vec
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [ ]:
initialDF_midCovid= pd.read_csv('Hansard_mid_covid.csv', sep=',', encoding='utf-16-le')
initialDF_midCovid.drop(['Publication', 'First Name', 'Last Name','Constituency', 'Province', 'Date', 'Time', 'Page'], axis=1)
initialDF_midCovid['Text'] = initialDF_midCovid['Text'].str.replace(r"^.*?Speaker,", ' ', regex=True).astype(str)

Preprocessing the text column using nltk library begins

In [ ]:

# !python -m pip install top2vec[sentence_encoders]


In [ ]:
midCovid_unguidedModel = Top2Vec(initialDF_midCovid['Text'].values, embedding_model='universal-sentence-encoder')
midCovid_unguidedModel.topic_words

In [ ]:
words, word_scores = midCovid_unguidedModel.similar_words(keywords=["workers"], keywords_neg=[], num_words=40)
for word, score in zip(words, word_scores):
    print(f"{word} {score}")

In [ ]:
topic_words, word_scores, topic_scores, topic_nums = midCovid_unguidedModel.search_topics(keywords=["labour"], num_topics=2)

In [ ]:
topic_words

Now let's check pre-covid 

In [10]:
# initialDF_preCovid= pd.read_csv('Hansard_preCovid.csv', sep=',', encoding='utf-16-le', error_bad_lines = False) 
# # may need to change separator to include skipped lines
# initialDF_preCovid.drop(['Publication', 'First Name', 'Last Name','Constituency', 'Province', 'Date', 'Time', 'Page'], axis=1) 

newline = []

textstring = ''
with open('Hansard_preCovid.csv', 'r', encoding='utf-16-le') as inp, open('newtextfile.txt', 'w') as out:
    for line in inp:
      newline.append(re.findall(r'"(.*)"', line))
      
temp = pd.Series(newline)

initialDF_preCovid = pd.DataFrame({'Text': temp})
clearWords = stopwords.words('english')
new_Stopwords = ['in', 'the', 'said', 'like','must', 'many', 'also']
clearWords.extend(new_Stopwords)
lemma= WordNetLemmatizer()
initialDF_preCovid['Text'] = initialDF_preCovid['Text'].astype(str).str.lower()
initialDF_preCovid['Text'] = initialDF_preCovid['Text'].str.replace("|".join([r"^.*?speaker,", r"^.*?chair," ]), ' ', regex=True)
initialDF_preCovid['text_tokens'] = initialDF_preCovid['Text'].apply(word_tokenize)
initialDF_preCovid['text_tokens'] = initialDF_preCovid['text_tokens'].apply(lambda x: [item for item in x if item not in clearWords])
initialDF_preCovid['text_tokens'] = initialDF_preCovid['text_tokens'].apply(lambda x: [lemma.lemmatize(item) for item in x])
# do we love me or what? isn't this join thing amazing?
initialDF_preCovid['text_tokens'] = initialDF_preCovid['text_tokens'].apply(lambda x:' '.join([item for item in x if len(item)>3]))

In [ ]:
preCovid_unguidedModel = Top2Vec(initialDF_preCovid['text_tokens'].values, embedding_model='universal-sentence-encoder')

In [12]:
preCovid_unguidedModel.topic_words

array([['immigration', 'immigrant', 'worker', 'nafta', 'labour',
        'refugee', 'citizenship', 'resident', 'caregiver', 'visa',
        'citizen', 'border', 'applicant', 'shortage', 'employment',
        'employer', 'disability', 'indigenous', 'temporary', 'process',
        'motion', 'parliamentary', 'wage', 'initiative', 'term',
        'employee', 'agreement', 'concern', 'skilled', 'foreign',
        'permanent', 'demand', 'ceta', 'significant', 'working',
        'conservative', 'claim', 'issue', 'consultant', 'system',
        'canadian', 'appeal', 'matter', 'provision', 'backlog',
        'liberal', 'asylum', 'work', 'economy', 'minister'],
       ['shortage', 'previous', 'immigration', 'debate', 'development',
        'nafta', 'total', 'indigenous', 'backlog', 'point', 'liberal',
        'filipino', 'economy', 'canadian', 'claim', 'province',
        'provincial', 'hope', 'believe', 'month', 'matter', 'enough',
        'construction', 'fact', 'broken', 'global', 'border', 'f

In [40]:
alpha, beta = preCovid_unguidedModel.similar_words(keywords=["foreign","labour"], keywords_neg=[], num_words=40)
for word, score in zip(alpha, beta):
    print(f"{word} {score}")

immigrant 0.5515805307206525
british 0.5200922804113817
immigration 0.4799257633885283
wage 0.4732187469352054
resident 0.46973550099469835
worker 0.4675052327334641
american 0.4650453734989143
international 0.46327797703190243
national 0.46190438988333526
employment 0.45419147782986585
employee 0.45350021684460906
canadian 0.4324903386632347
expenditure 0.4315706811459571
broken 0.4294647734017978
government 0.41818938770041997
conservative 0.41649454630841926
year 0.4164675533382584
local 0.41450124710440284
country 0.4133913624072004
change 0.41257050277554924
different 0.409711326085784
economy 0.4076529937596496
nation 0.40698500586686226
work 0.40370080855711643
investment 0.4033272379705356
citizen 0.40329748379304
shortage 0.40192171926918646
citizenship 0.40077371387712263
riding 0.3995600766024652
child 0.39836679973270783
people 0.3981921291543734
opportunity 0.39742322751764286
employer 0.3972850597075456
issue 0.395564882579488
part 0.39101333957612155
home 0.3886211088481

In [39]:
topic_words, topic_scores

([array(['immigration', 'immigrant', 'worker', 'nafta', 'labour', 'refugee',
         'citizenship', 'resident', 'caregiver', 'visa', 'citizen',
         'border', 'applicant', 'shortage', 'employment', 'employer',
         'disability', 'indigenous', 'temporary', 'process', 'motion',
         'parliamentary', 'wage', 'initiative', 'term', 'employee',
         'agreement', 'concern', 'skilled', 'foreign', 'permanent',
         'demand', 'ceta', 'significant', 'working', 'conservative',
         'claim', 'issue', 'consultant', 'system', 'canadian', 'appeal',
         'matter', 'provision', 'backlog', 'liberal', 'asylum', 'work',
         'economy', 'minister'], dtype='<U13')],
 array([0.24572332]))

post covid

In [2]:
#Converting csv to txt file for ease of processing
newline = []

textstring = ''
with open('Hansard_postCovid.csv', 'r', encoding='utf-16-le') as inp, open('newtextfile.txt', 'w') as out:
    for line in inp:
      newline.append(re.findall(r'"(.*)"', line))
      
temp = pd.Series(newline)

initialDF_postCovid = pd.DataFrame({'Text': temp})
clearWords = stopwords.words('english')
new_Stopwords = ['in', 'the', 'said', 'like','must', 'many', 'also']
clearWords.extend(new_Stopwords)
lemma= WordNetLemmatizer()

initialDF_postCovid['Text'] = initialDF_postCovid['Text'].astype(str).str.lower()
initialDF_postCovid['Text'] = initialDF_postCovid['Text'].str.replace("|".join([r"^.*?speaker,", r"^.*?chair," ]), ' ', regex=True)
initialDF_postCovid['text_tokens'] = initialDF_postCovid['Text'].apply(word_tokenize)
initialDF_postCovid['text_tokens'] = initialDF_postCovid['text_tokens'].apply(lambda x: [item for item in x if item not in clearWords])
initialDF_postCovid['text_tokens'] = initialDF_postCovid['text_tokens'].apply(lambda x: [lemma.lemmatize(item) for item in x])
# do we love me or what? isn't this join thing amazing?
initialDF_postCovid['text_tokens'] = initialDF_postCovid['text_tokens'].apply(lambda x:' '.join([item for item in x if len(item)>3]))


In [ ]:

postCovid_unguidedModel = Top2Vec(initialDF_postCovid['text_tokens'].values, embedding_model='universal-sentence-encoder')


In [ ]:

postCovid_unguidedModel.topic_words

In [23]:
alpha, beta = postCovid_unguidedModel.similar_words(keywords=["temporary", "labour"], keywords_neg=[], num_words=25)
for word, score in zip(alpha, beta):
    print(f"{word} {score}")

permanent 0.5147940949139227
term 0.47384104545421524
election 0.4663639861164546
economy 0.42325586153495187
worker 0.4139296705635098
opportunity 0.39194256501063085
shortage 0.38861753023836454
time 0.3833539235873233
able 0.38201068492591184
economic 0.37937424423252303
cost 0.3713025436315296
long 0.3698403539823836
issue 0.35968442293537384
immigrant 0.3574537160113743
working 0.3484637988296785
government 0.34830348694544594
resident 0.34794954056206967
riding 0.34579957302582687
immigration 0.3452633231057951
conservative 0.3426807328002873
change 0.34033209593929203
small 0.33651360210393566
work 0.33589125663835895
crisis 0.3348128490982276
child 0.334699688208134
